In [2]:
import pandas as pd
import numpy as np
import os
import sys
import ast

In [3]:
mod_path = '/Users/ishareef7/Springboard/Capstone2'
sys.path.append(mod_path)
from src.features import build_dataset

In [4]:
features = build_dataset.get_features()
echonest = build_dataset.get_echonest()
tracks = build_dataset.get_tracks()
genres = build_dataset.get_genres()

In [4]:
tracks['track','genres'] = tracks['track','genres'].apply(lambda x: ast.literal_eval(x))

In [5]:
idx = pd.IndexSlice
track_info = tracks.loc[:, idx[['set','track'], ['split', 'subset','genre_top','genres']]].copy()
track_info.columns = track_info.columns.droplevel(0)

In [6]:
null_mask = track_info['genre_top'].isnull().copy()
null_genres = track_info[null_mask]
empty_mask = [len(x)>0 for x in  null_genres['genres']]
null_genres = null_genres[empty_mask]

In [7]:
genre_dict = dict(genres['title'])

In [8]:
null_genres['genre_top'] = null_genres['genres'].apply(lambda x: x[0])
top_genres = [ genre_dict[x] for x in null_genres['genre_top'].values]

In [9]:
echonest_features = echonest.loc[:, idx[['audio_features', 'temporal_features'],:]]

In [10]:
#Flatten Indexes
features.columns = ['_'.join(col).strip('_') for col in features.columns.values]
echonest_features.columns = ['_'.join(col).rstrip('_') for col in echonest_features.columns.values]

In [11]:
track_info.loc[null_genres.index, 'genre_top']  = top_genres

In [12]:
top_genre_dict = dict(zip(genres['title'], genres['top_level']))
track_info['genre_top'] = track_info['genre_top'].map(top_genre_dict)

In [13]:
dataset_echonest = features.join(echonest_features, how = 'inner').join(track_info, how = 'inner')
dataset_no_echnoest = features.join(track_info, how = 'inner')